In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import requests
import json

# API Key
from config import gkey

In [2]:
# Import cities info from csv
cities_df = pd.read_csv("../Resources/KC_Metro_Cities.csv")
cities_df.head()

,Cities,State
0,Belton,Missouri
1,Gardner,Kansas
2,Gladstone,Missouri
3,Grandview,Missouri
4,Independence,Missouri


In [3]:
# add columns for restaurant name, restaurant address, rating
cities_df["Latitude"]=""
cities_df["Longitude"]=""
cities_df["Place Name"]=""
cities_df["Place Address"]=""
cities_df["Place Rating"]=""
cities_df["Place Latitude"]=""
cities_df["Place Longitude"]=""
cities_df.head()

,Cities,State,Latitude,Longitude,Place Name,Place Address,Place Rating,Place Latitude,Place Longitude
0,Belton,Missouri,,,,,,,
1,Gardner,Kansas,,,,,,,
2,Gladstone,Missouri,,,,,,,
3,Grandview,Missouri,,,,,,,
4,Independence,Missouri,,,,,,,


In [4]:
# create params dictionary to be updated with new city each iteration
params = {"key": gkey}

# loop through cities_df and run lat/lon search for each city
for index, row in cities_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    
    city = row["Cities"]
    state = row["State"]
    
    # update address key value
    params["address"] = f"{city},{state}"
    
    # make request
    cities_lat_lon = requests.get(base_url, params=params)
#     print(cities_lat_lon.url)
    
    # convert to json
    cities_lat_lon = cities_lat_lon.json()
    
    cities_df.loc[index, "Latitude"]=cities_lat_lon["results"][0]["geometry"]["location"]["lat"]
    cities_df.loc[index, "Longitude"]=cities_lat_lon["results"][0]["geometry"]["location"]["lng"]

#visualize to confirm
cities_df.head()

,Cities,State,Latitude,Longitude,Place Name,Place Address,Place Rating,Place Latitude,Place Longitude
0,Belton,Missouri,38.812,-94.5319,,,,,
1,Gardner,Kansas,38.8108,-94.9272,,,,,
2,Gladstone,Missouri,39.2039,-94.5547,,,,,
3,Grandview,Missouri,38.8858,-94.533,,,,,
4,Independence,Missouri,39.0911,-94.4155,,,,,


In [5]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "gym",
    "key": gkey
}

# use latitude/longitude to identify chinese retaurants
for index, row in cities_df.iterrows():
#     print(index)
#     get lat/lon from df
    lat = row["Latitude"]
    lon = row["Longitude"]
    
#     change location each iteration while 
#     leaving original params in place
    params["location"] = f"{lat},{lon}"
    
#     use the serach term "Chines Restaurant" and lat/lon
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
#     make request and print url
    response = requests.get(base_url, params=params)
    
#     print(response.url)

#     convert to json
    response = response.json()
#     print(json.dumps(response, indent=4, sort_keys=True))

#     since some data may be missing, use try/except to skip any missing
#     data point
#     loop through to gather 5 indices
    city_name = []
    state_name = []
    city_lat = []
    city_lon = []
    name = []
    address = []
    rating = []
    place_lat = []
    place_lon = []
    i = 0
    for responses in response:
        while i < 5:
            try:
                city_name.append(row["Cities"])
                state_name.append(row["State"])
                city_lat.append(lat)
                city_lon.append(lon)
                name.append(response["results"][i]["name"])
                address.append(response["results"][i]["vicinity"])
                rating.append(response["results"][i]["rating"])
                place_lat.append(response["results"][i]["geometry"]["location"]["lat"])
                place_lon.append(response["results"][i]["geometry"]["location"]["lng"])
            except (KeyError, IndexError):
                print("Missing field/result... skipping.")
        
            i+=1
#     if index = 0 create a new dataframe        
    if index == 0:
        gyms_df = pd.DataFrame({"City":city_name,
                                             "State":state_name,
                                             "Latitude":city_lat,
                                             "Longitude":city_lon,
                                             "Place Name":name,
                                             "Place Address":address,
                                             "Place Rating":rating,
                                             "Place Latitude":place_lat,
                                             "Place Longitude":place_lon})
# otherwise create a new dataframe and then append the new dataframe to [0] df
    else:
        other_gyms_df = pd.DataFrame({"City":city_name,
                                        "State":state_name,
                                        "Latitude":city_lat,
                                        "Longitude":city_lon,
                                        "Place Name":name,
                                        "Place Address":address,
                                        "Place Rating":rating,
                                        "Place Latitude":place_lat,
                                        "Place Longitude":place_lon})
        gyms_df = gyms_df.append(other_gyms_df).reset_index(drop=True)
        

In [6]:
gyms_df

,City,State,Latitude,Longitude,Place Name,Place Address,Place Rating,Place Latitude,Place Longitude
0,Belton,Missouri,38.811953,-94.531898,Life Time Fitness,"16851 West 90th Street, Lenexa",3.9,38.964188,-94.781214
1,Belton,Missouri,38.811953,-94.531898,24 Hour Fitness,"12075 Metcalf Avenue, Overland Park",3.3,38.909800,-94.666277
2,Belton,Missouri,38.811953,-94.531898,New Century Fieldhouse,"551 New Century Parkway, New Century",4.5,38.819151,-94.903294
3,Belton,Missouri,38.811953,-94.531898,Genesis Health Club,"13370 South Blackfoot Drive, Olathe",3.2,38.885470,-94.766061
4,Belton,Missouri,38.811953,-94.531898,68's INSIDE SPORTS,"11301 West 88th Street, Overland Park",3.9,38.967876,-94.716948
5,Gardner,Kansas,38.810841,-94.927187,Life Time Fitness,"16851 West 90th Street, Lenexa",3.9,38.964188,-94.781214
6,Gardner,Kansas,38.810841,-94.927187,24 Hour Fitness,"12075 Metcalf Avenue, Overland Park",3.3,38.909800,-94.666277
7,Gardner,Kansas,38.810841,-94.927187,New Century Fieldhouse,"551 New Century Parkway, New Century",4.5,38.819151,-94.903294
8,Gardner,Kansas,38.810841,-94.927187,Genesis Health Club,"13370 South Blackfoot Drive, Olathe",3.2,38.885470,-94.766061
9,Gardner,Kansas,38.810841,-94.927187,68's INSIDE SPORTS,"11301 West 88th Street, Overland Park",3.9,38.967876,-94.716948


In [7]:
gyms_df.to_csv("../Output/gyms.csv")

In [8]:
import sqlite3

In [9]:
conn = sqlite3.connect('../Output/places_db.sqlite')

In [10]:
gyms_df.to_sql('gyms', conn, if_exists='replace', index=False)

/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2130: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype)


In [11]:
pd.read_sql('select * from gyms', conn)

,City,State,Latitude,Longitude,Place Name,Place Address,Place Rating,Place Latitude,Place Longitude
0,Belton,Missouri,38.811953,-94.531898,Life Time Fitness,"16851 West 90th Street, Lenexa",3.9,38.964188,-94.781214
1,Belton,Missouri,38.811953,-94.531898,24 Hour Fitness,"12075 Metcalf Avenue, Overland Park",3.3,38.909800,-94.666277
2,Belton,Missouri,38.811953,-94.531898,New Century Fieldhouse,"551 New Century Parkway, New Century",4.5,38.819151,-94.903294
3,Belton,Missouri,38.811953,-94.531898,Genesis Health Club,"13370 South Blackfoot Drive, Olathe",3.2,38.885470,-94.766061
4,Belton,Missouri,38.811953,-94.531898,68's INSIDE SPORTS,"11301 West 88th Street, Overland Park",3.9,38.967876,-94.716948
5,Gardner,Kansas,38.810841,-94.927187,Life Time Fitness,"16851 West 90th Street, Lenexa",3.9,38.964188,-94.781214
6,Gardner,Kansas,38.810841,-94.927187,24 Hour Fitness,"12075 Metcalf Avenue, Overland Park",3.3,38.909800,-94.666277
7,Gardner,Kansas,38.810841,-94.927187,New Century Fieldhouse,"551 New Century Parkway, New Century",4.5,38.819151,-94.903294
8,Gardner,Kansas,38.810841,-94.927187,Genesis Health Club,"13370 South Blackfoot Drive, Olathe",3.2,38.885470,-94.766061
9,Gardner,Kansas,38.810841,-94.927187,68's INSIDE SPORTS,"11301 West 88th Street, Overland Park",3.9,38.967876,-94.716948
